In [1]:
import inspect
import numpy as np
import pandas

def list_size(variable, callers_local_vars):
    list_size_recursive(' # ', variable, callers_local_vars)

def list_size_recursive(header, variable, callers_local_vars):
    if type(variable) == list or type(variable) == pandas.core.series.Series:
        print(header, 'len(', str([k for k, v in callers_local_vars if v is variable][0]), ') = ', len(variable), sep='')
        if(len(variable) > 0):
            header += '  '
            list_size_recursive(header, variable[0], callers_local_vars)

def nparray_size(variable, callers_local_vars):
    if(type(variable) == np.ndarray):
        print(' # ', str([k for k, v in callers_local_vars if v is variable][0]), '.shape = ', variable.shape, sep='')

def see(variable, all=0):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    print('# ', str([k for k, v in callers_local_vars if v is variable][0]), ' (', type(variable), ')', sep='', end='')
    if all or (type(variable) != list and type(variable) != np.ndarray and type(variable) != pandas.core.series.Series):
        print(' = ', variable, sep='')
    else:
        print('')
    list_size(variable, callers_local_vars)
    nparray_size(variable, callers_local_vars)


In [2]:
import inspect
import numpy as np
import pandas

def list_size(variable, callers_local_vars):
    list_size_recursive(' # ', variable, callers_local_vars)

def list_size_recursive(header, variable, callers_local_vars):
    if type(variable) == list or type(variable) == pandas.core.series.Series:
        print(header, 'len(', str([k for k, v in callers_local_vars if v is variable][0]), ') = ', len(variable), sep='')
        if(len(variable) > 0):
            header += '  '
            list_size_recursive(header, variable[0], callers_local_vars)

def nparray_size(variable, callers_local_vars):
    if(type(variable) == np.ndarray):
        print(' # ', str([k for k, v in callers_local_vars if v is variable][0]), '.shape = ', variable.shape, sep='')

def see(variable, all=0):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    print('# ', str([k for k, v in callers_local_vars if v is variable][0]), ' (', type(variable), ')', sep='', end='')
    if all or (type(variable) != list and type(variable) != np.ndarray and type(variable) != pandas.core.series.Series):
        print(' = ', variable, sep='')
    else:
        print('')
    list_size(variable, callers_local_vars)
    nparray_size(variable, callers_local_vars)

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

attrNames = ["sepal length", "sepal width", "petal length", "petal width"]

dataList = []
classLabelsList = []
f = open('iris.data', 'r')
for line in f:
    line = line.rstrip()
    elems = line.split(',')
    if len(elems) > 1:
        dataList.append(elems[:-1])
        classLabelsList.append(elems[-1])
f.close()

X = np.float_(np.array(dataList))

# convert y to one hot
labels = np.unique(classLabelsList)

def label2onehot(label):
    return np.array([1 if label == elem else 0 for elem in labels])

y = np.array([label2onehot(label) for label in classLabelsList])



In [21]:
import numpy as np

def softmax(a):
    return np.exp(a) / np.sum(np.exp(a))

def softmax_nn(x_with_bias, W):
    see(x_with_bias)
    see(W)
    return softmax(np.matmul(x_with_bias.transpose(), W))

In [31]:
def grad(x_with_bias, y, W):
    error = softmax_nn(x_with_bias, W) - y
    return np.matmul(x_with_bias.transpose(), error) / x_with_bias.shape[0]

In [32]:
def update(x_with_bias, y, W, learning_rate):
    g = grad(x_with_bias, y, W)
    delta_W = - learning_rate * g
    return W + delta_W

In [33]:
def train(x, y, learning_rate, iterNum):    
    x_with_bias = np.c_[x, np.ones(x.shape[0])]
    W = np.random.random((y.shape[1], x_with_bias.shape[1]))
    loss_dynamics = []
    for _ in range(iterNum):
        W = update(W, x_with_bias, y, learning_rate)
        W_dynamics.append(w)
    return W, W_dynamics

In [34]:
learning_rate = 0.01
iterNum = 10
trained_W, W_dynamics = train(X, y, learning_rate, iterNum)

# x_with_bias (<class 'numpy.ndarray'>)
 # x_with_bias.shape = (3, 5)
# W (<class 'numpy.ndarray'>)
 # W.shape = (150, 3)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 150 is different from 3)

In [35]:
softmax_nn(X, trained_W)

NameError: name 'trained_W' is not defined

In [30]:
trained_W

NameError: name 'trained_W' is not defined